In [ ]:
import os
import csv
import cv2
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout, ELU
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras import backend as K

def load_driving_log(path='./data', split='\\'):
    csv = pd.read_csv(os.path.join(path, 'driving_log.csv'), header=None,
                          names=['center','left','right','angle','throttle','break','speed'])

    csv.center = csv.center.str.split(split).str[-1]
    csv.left = csv.left.str.split(split).str[-1]
    csv.right = csv.right.str.split(split).str[-1]
    return csv

def generator(samples, path='./data', batch_size=32):
    num_samples = len(samples)
    print(num_samples)
    while 1:
        shuffle(samples)
    
        for offset in range(0, num_samples, batch_size):

            images = []
            angles = []
            batch_samples = samples[offset:offset + batch_size]

            for batch_sample in batch_samples:
                name = os.path.join(path, 'IMG', batch_sample[0])
                image = cv2.resize(cv2.imread(name), (200, 100), interpolation=cv2.INTER_AREA)
                image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
                images.append(image)
                angle = float(batch_sample[3])
                angles.append(angle)

            yield shuffle(np.array(images), np.array(angles))

In [3]:
pwd

'/Users/jaydenmilton/Documents/Self-Driving-Car-Engineer-Nanodegree-Program/CarND-Behavioral-Cloning-P3'

In [38]:
#import data 

lines = []
file_name = "/Users/jaydenmilton/Documents/Self-Driving-Car-Engineer-Nanodegree-Program/CarND-Behavioral-Cloning-P3/data/driving_log.csv"

with open(file_name) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []

for line in lines:
    image = cv2.imread(line[0])
#assert image is not None
    filename = source_path.split('\\')[-1]
    current_path = '/Users/jaydenmilton/Documents/Self-Driving-Car-Engineer-Nanodegree-Program/CarND-Behavioral-Cloning-P3/data/IMG/' + filename
    image = cv2.imread(current_path)
    if image is None:
        print("Image path incorrect: ", current_path)
        continue  # skip adding these rows in the for loop

    images.append(image)
    measurement= float(line[3])
    measurements.append(measurement)

X_train = np.array(images)
y_train = np.array(measurements)


from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape=(160,320,3)))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2)

model.save('model.h5')


SyntaxError: EOL while scanning string literal (<ipython-input-38-4da2c52dc78c>, line 17)

In [7]:
with open("/Users/jaydenmilton/Documents/Self-Driving-Car-Engineer-Nanodegree-Program/CarND-Behavioral-Cloning-P3/data/driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
for line in lines:
    filename = line[0]
    current_center = cv2.imread(filename)
    images.append(current_center)
    
print(current_center.shape)
print(type(current_center))

(160, 320, 3)
<class 'numpy.ndarray'>


In [40]:
import csv 
import cv2
import numpy as np
lines = []
with open('/Users/jaydenmilton/Documents/Self-Driving-Car-Engineer-Nanodegree-Program/CarND-Behavioral-Cloning-P3/data/driving_log.csv')as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
for line in lines:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = '../data/IMG' + filename
    image = cv2.imread(current_path)
    images.append(image)

    try:
        measurement = float(line[3])
        measurements.append(measurement)
    except:
        # maybe print something like this so blank values are not missed out
        # print("Error!! Trying to convert %s to float" % line[3])
        print(line[3])
    
X_train = np.array(images)
y_train = np.array(measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape= (160,320,3)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle= True, nb_epoch=7)
model.save('model.h5')

ValueError: Error when checking model input: expected flatten_input_9 to have 4 dimensions, but got array with shape (6853, 1)

In [ ]:
pwd

In [41]:
driving_log = load_driving_log()
driving_log.info()

NameError: name 'load_driving_log' is not defined

In [ ]:
driving_log.head()

In [ ]:
train_samples, validation_samples = train_test_split(driving_log, test_size=0.2)

In [ ]:
print('Driving log length:', len(driving_log))
print('Training samples length:', len(train_samples))
print('Validation samples length:', len(validation_samples))

In [ ]:
print(list(train_samples))
train_generator = generator(list(train_samples), batch_size=32)
validation_generator = generator(list(validation_samples), batch_size=32)

In [ ]:
# 1. define basic model
model = Sequential()
# 6. add cropping
model.add(Cropping2D(cropping=((70, 25), (0, 0)), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: x / 255 - 0.5, trainable=False))
model.add(Convolution2D(24, 5, 5, subsample=(2, 2), border_mode="valid"))
model.add(ELU())
model.add(Convolution2D(36, 5, 5, subsample=(2, 2), border_mode="valid"))
model.add(ELU())
model.add(Convolution2D(48, 5, 5, subsample=(2, 2), border_mode="valid"))
model.add(ELU())
model.add(Convolution2D(64, 3, 3, border_mode="valid"))
model.add(ELU())
model.add(Convolution2D(64, 3, 3, border_mode="valid"))
model.add(Dropout(0.2))
model.add(ELU())
model.add(Flatten())
model.add(Dense(100))
model.add(ELU())
model.add(Dense(50))
model.add(ELU())
model.add(Dense(10))
model.add(ELU())
model.add(Dense(1, trainable=False))

model.summary()

In [43]:
import pandas as pd
sample_data = pd.read_csv('/Users/jaydenmilton/Documents/Self-Driving-Car-Engineer-Nanodegree-Program/CarND-Behavioral-Cloning-P3/data/driving_log.csv')
sample_data.head()

,/Users/jaydenmilton/Documents/IMG/center_2018_04_26_19_06_59_225.jpg,/Users/jaydenmilton/Documents/IMG/left_2018_04_26_19_06_59_225.jpg,/Users/jaydenmilton/Documents/IMG/right_2018_04_26_19_06_59_225.jpg,0,0.1,0.2,0.8402984
0,/Users/jaydenmilton/Documents/IMG/center_2018_...,/Users/jaydenmilton/Documents/IMG/left_2018_04...,/Users/jaydenmilton/Documents/IMG/right_2018_0...,0.00,0.000000,0,0.828530
1,/Users/jaydenmilton/Documents/IMG/center_2018_...,/Users/jaydenmilton/Documents/IMG/left_2018_04...,/Users/jaydenmilton/Documents/IMG/right_2018_0...,0.00,0.000000,0,0.816926
2,/Users/jaydenmilton/Documents/IMG/center_2018_...,/Users/jaydenmilton/Documents/IMG/left_2018_04...,/Users/jaydenmilton/Documents/IMG/right_2018_0...,0.00,0.000000,0,0.803865
3,/Users/jaydenmilton/Documents/IMG/center_2018_...,/Users/jaydenmilton/Documents/IMG/left_2018_04...,/Users/jaydenmilton/Documents/IMG/right_2018_0...,-0.35,0.417969,0,1.473115
4,/Users/jaydenmilton/Documents/IMG/center_2018_...,/Users/jaydenmilton/Documents/IMG/left_2018_04...,/Users/jaydenmilton/Documents/IMG/right_2018_0...,-0.65,0.750696,0,2.405843


In [44]:
import seaborn as sns
%matplotlib inline
sns.distplot(sample_data.steering);
plt.savefig("steering1.png")
plt.show()

AttributeError: 'DataFrame' object has no attribute 'steering'

In [48]:
import os
import pandas as pd
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, MaxPooling2D, Conv2D, Lambda
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import shuffle

import tensorflow as tf

epochs = 50
batch_size = 128
dataset_dir = "/Users/jaydenmilton/Documents/Self-Driving-Car-Engineer-Nanodegree-Program/data/IMG"
image_columns = 32
image_rows = 16
image_channels = 1
side_shift = 0.3


def preproccess_image(image):
    image = (cv2.cvtColor(image, cv2.COLOR_BGR2HSV))[:, :, 1]
    image = image.reshape(160, 320, 1)
    image = cv2.resize(image, (image_columns, image_rows))
    return image


def prepare(data):
    x, y = [], []

    for i in range(len(data)):
        line_data = data.iloc[i]
        y_steer = line_data['steering']
        path_center = line_data['center'].strip()
        path_left = line_data['left'].strip()
        path_right = line_data['right'].strip()

        for path, shift in [(path_center, 0), (path_left, side_shift), (path_right, -side_shift)]:
            # read image
            image_path = os.path.join(dataset_dir, path)
            image = cv2.imread(image_path)

            # preprocess image
            image = preproccess_image(image)

            # add image
            x.append(image)
            y.append(y_steer + shift)

            # add flipped image
            image = image[:, ::-1]
            x.append(image)
            y.append(-(y_steer + shift))

    return np.array(x).astype('float32'), np.array(y).astype('float32')


def model():
    model = Sequential()
    model.add(Lambda(lambda x: x / 127.5 - 1.0, input_shape=(image_rows, image_columns, image_channels)))
    model.add(Conv2D(2, 3, 3, border_mode='valid', activation='elu'))
    model.add(MaxPooling2D((4, 4), (4, 4), 'valid'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1))

    return model


if __name__ == '__main__':
    print("Loading images...")

    data = pd.read_csv(os.path.join(dataset_dir, "/Users/jaydenmilton/Documents/Self-Driving-Car-Engineer-Nanodegree-Program/data/driving_log.csv"))

    X_train, y_train = prepare(data)
    X_train, y_train = shuffle(X_train, y_train)
    X_train = np.expand_dims(X_train, axis=3)

    model = model()
    model.compile('adam', 'mean_squared_error', ['mean_squared_error'])
    checkpoint = ModelCheckpoint("model.h5", monitor='val_mean_squared_error', verbose=1,
                                  save_best_only=True, mode='min')
    early_stop = EarlyStopping(monitor='val_mean_squared_error', min_delta=0.0001, patience=4,
                                verbose=1, mode='min')
    model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epochs, verbose=1,
                      callbacks=[checkpoint, early_stop], validation_split=0.15, shuffle=True)

    print("Saving model...")
    with open("model.json", 'w') as outfile:
        outfile.write(model.to_json())

    print("Finished.")

Loading images...


FileNotFoundError: File b'/Users/jaydenmilton/Documents/Self-Driving-Car-Engineer-Nanodegree-Program/data/driving_log.csv' does not exist

In [ ]:
### data preprocessing 

X_train = np.array(images)
y_train = np.array(measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda

model = Sequential()
#first normalize the data image
model.add = (Lambda(lambda x:x /255.0), input_shape = (160,320,3))
#model.add = (Lambda(lambda x:x /255.0 -0.0), input_shape = (160,320,3))


model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer="adam")
#reducing the epoch, since the data had been overfitting 
model.fit(X_train, y_train, validation_split=0.2,shuffle=True, nb_epoch-5)




In [ ]:
# data augrementation 
#numpy/cv2 has that 
#https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#flip


import numpy as np
image_flipped = np.fliplr(image)
measurement_flipped = -measurement



In [ ]:
# multi-camera training 


with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            steering_center = float(row[3])

            # create adjusted steering measurements for the side camera images
            correction = 0.2 # this is a parameter to tune
            steering_left = steering_center + correction
            steering_right = steering_center - correction

            # read in images from center, left and right cameras
            path = "..." # fill in the path to your training IMG directory
            img_center = process_image(np.asarray(Image.open(path + row[0])))
            img_left = process_image(np.asarray(Image.open(path + row[1])))
            img_right = process_image(np.asarray(Image.open(path + row[2])))

            # add images and angles to data set
            car_images.extend(img_center, img_left, img_right)
            steering_angles.extend(steering_center, steering_left, steering_right)

In [ ]:
#copping the image 

from keras.models import Sequential, Model
from keras.layers import Cropping2D
import cv2

# set up cropping2D layer
model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))
...

In [ ]:
#Nvida network 
#fewer epoch steps that makes better 

In [ ]:
#visualizing loss 

from keras.models import Model
import matplotlib.pyplot as plt

history_object = model.fit_generator(train_generator, samples_per_epoch =
    len(train_samples), validation_data = 
    validation_generator,
    nb_val_samples = len(validation_samples), 
    nb_epoch=5, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

In [50]:
#Generators

import os
import csv

samples = []
with open('./driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

import cv2
import numpy as np
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

            '''
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: x/127.5 - 1.,
        input_shape=(ch, row, col),
        output_shape=(ch, row, col)))
model.add(... finish defining the rest of your model architecture here ...)

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch= /
            len(train_samples), validation_data=validation_generator, /
            nb_val_samples=len(validation_samples), nb_epoch=3)

"""
If the above code throw exceptions, try 
model.fit_generator(train_generator, steps_per_epoch= len(train_samples),
validation_data=validation_generator, validation_steps=len(validation_samples), epochs=5, verbose = 1)
"""

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-50-b1fca75a62ac>, line 62)